Reference = https://medium.com/daangn/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9C%BC%EB%A1%9C-%EB%8F%99%EB%84%A4%EC%83%9D%ED%99%9C-%EA%B2%8C%EC%8B%9C%EA%B8%80-%ED%95%84%ED%84%B0%EB%A7%81%ED%95%98%EA%B8%B0-263cfe4bc58d

In [1]:
import pandas as pd
import pickle

# Process
 ### BERT Pretrain
 - 스팸 분류를 위해서 기존 유튜브의 일반적인 댓글들에 대해 pretrain이 필요  
 -- Youtube 댓글 크롤링 (완료)  
 -- 댓글 전처리 & 토크나이즈 (wordpiece or sentencepiece or mecab이용한 bert tokenize-electra)  
 -- 학습을 위해선 문장 2개를 '\t'로 붙이고, 마지막에 '\n'으로 데이터를 구성 (?)  
 -- 학습
 ### Dataset 구축
 - 필터링을 위한 데이터 구축
 -- 당근마켓의 경우 실제 필요한 게시글 7290 / 필요 없는 게시글 710 라벨링  
 -- 8000개 학습 데이터 중 2000개를 테스트 데이터로 활용  
 -- sklearn을 통해 지표확인(recall 높게 유지하면서 precision 높이는 것 - 맞는 게시글을 미노출 시키는 경우 방지)
 

## 스팸 데이터셋 구축

In [3]:
import os

path_dir = './'
file_list = os.listdir(path_dir)

In [9]:
#크롤링된 데이터 불러오기
df_all = pd.DataFrame()
for file in file_list:
    if 'trending_video' in file:
        temp_df = pd.read_pickle(file)
        df_all = pd.concat([df_all, temp_df])
        
#중복, 결측치 제거
df_all.dropna(inplace=True)
df_all.drop_duplicates(inplace=True)

In [10]:
review_list = list(df_all['review'])

In [11]:
len(review_list)

49672

In [12]:
def get_KeywordDF(keyword):
    temp_list = []    
    for review in review_list:
        if keyword in review:
            temp_list.append(review)

    return list(set(temp_list))

In [13]:
spam_list = []

### 키워드로 스팸 찾기
- 댓글을 보다보면 일관된 단어들이 보인다.  
- 하나의 키워드로 필터링을 하고, 필터링 된 스팸에서 다른 키워드를 찾아 이전의 키워드에서 찾아내지 못한 스팸을 찾아낸다.

In [14]:
spam_1 = get_KeywordDF('채널')
spam_1

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '@하루10,000원 받아가세요 채널클릭 한번에 1억 줄 수 없다면 꺼지세여',
 '#이찬원 #유튜브채널\n2020년 한해는 찬또배기 이찬원님  저에게 주는 기쁨과 즐거움 행복함  넘 고맙고  감사합니다~항상 찬스들이 곁에서 응원합니다',
 '진정성이 너무 무시되면 저질채널이 될수있습니다 좀더 신경써주셍_',
 '꼰대희채널은 밥묵자가진리!\n잘보갑니다!',
 '원장님도 asmr 채널 만들어주세여 ㅠㅠ 목소리도 연기력도 대박\U0001f97a',
 '@하루1만원씩 챙겨가세요 채널클릭 관종2',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '권재관씨 개그 채널 도전하세요~\n구독하고 보고 싶습니다~\n어려운 시기에 개그보고 \n힘든일 잊고 싶고 힘내게 되요~\n진짜 재밌었는데',
 '찬원님  유튜버 채널  개설  진심으로 추카추카   합니다   영웅님팬으로써  형이  사랑하는 찬원님  앞으로  구독자가  빠르게  빠르게 늘어나도록 진심으로 바라겠어요  저도 구독했어요 앞으로도 자주 들려서  찬원님 소식 듣겠습니다   항상 건강잘   챙기시구요',
 '뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '언니 유튜브 평생 해주세요 진짜 ... 레전드 채널 찾았움ㅠㅠ💖💖',
 '@네이버페이로5분만에 진짜 만원벌기 진용진 님 채널 다녀오셔요',
 '님 괜찮아여 힘들어도 힘내요~~~힘들면 쉬어도 되요 괜찮아요 저 이미 이 채널 있는 걸로 만족해요 힘들면 댓글로 말해주세여ㅎㅎ다음에 영상에서 봐여',
 '제가 잘 몰라서 그러는데 자기 채널에서 돈 받아가라는 사람들 채널 주인이 특정 텍스트 설정해서 그 텍스트는 댓글 입력 못하는거 없나요?',
 '근데 대단한건 런닝맨이 여기 채널도 그렇고 스브스 토렌트도 그렇고 런닝맨 관련 영상만 올리면 인기 급

### 채널명으로 필터링
- 대체적으로채널의 주인에 대한 호칭(채널명, 혹은 별명)이 포함된 댓글은 스팸이 아닌 경우가 많다.  
- 대신 몇몇 스팸들의 경우, 

In [18]:
temp_spam = []
for spam in spam_1:
    if '이찬원' in spam:
        pass
    elif '찬원' in spam:
        pass
    elif '원숙' in spam:
        pass
    elif '꼰대희' in spam:
        pass
    elif '대희' in spam:
        pass
    elif '권재관' in spam:
        pass
    elif '재관' in spam:
        pass
    elif '희극인' in spam:
        pass
    elif '감스트' in spam:
        pass
    elif '다정한 부부' in spam:
        pass
    elif '개그맨' in spam:
        pass
    elif 'RC' in spam:
        pass
    elif 'F5' in spam:
        pass
    elif '육아대디' in spam:
        pass
    elif '토모' in spam:
        pass
    elif '올리버' in spam:
        pass
    else:
        temp_spam.append(spam)

In [19]:
temp_spam

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '@하루10,000원 받아가세요 채널클릭 한번에 1억 줄 수 없다면 꺼지세여',
 '진정성이 너무 무시되면 저질채널이 될수있습니다 좀더 신경써주셍_',
 '원장님도 asmr 채널 만들어주세여 ㅠㅠ 목소리도 연기력도 대박\U0001f97a',
 '@하루1만원씩 챙겨가세요 채널클릭 관종2',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '언니 유튜브 평생 해주세요 진짜 ... 레전드 채널 찾았움ㅠㅠ💖💖',
 '@네이버페이로5분만에 진짜 만원벌기 진용진 님 채널 다녀오셔요',
 '님 괜찮아여 힘들어도 힘내요~~~힘들면 쉬어도 되요 괜찮아요 저 이미 이 채널 있는 걸로 만족해요 힘들면 댓글로 말해주세여ㅎㅎ다음에 영상에서 봐여',
 '제가 잘 몰라서 그러는데 자기 채널에서 돈 받아가라는 사람들 채널 주인이 특정 텍스트 설정해서 그 텍스트는 댓글 입력 못하는거 없나요?',
 '근데 대단한건 런닝맨이 여기 채널도 그렇고 스브스 토렌트도 그렇고 런닝맨 관련 영상만 올리면 인기 급상승이 바로 돼냐 역시 대한민국을 대표하는 예능이다',
 '내 채널에 상해람보 있음 한번 봐보셈',
 '@이벤트알려주는채널 ㅈㄹ ㄴㄴ',
 '엇 저랑 아이디 똑같으심ㅋㅋㅋㅋㅋ너무 공감이요ㅜㅜ제 2의 개콘 채널이 되었으면..',
 '@이벤트알려주는채널 그만좀해라 넌',
 '지 채널 홍보를 왜 여기서해',
 '제 채널들어와봐용\n\n공짜(돈/캐시/깊티)받아가세요\n피해 ㄴㄴㄴ!',
 '@김수빈 글쓴이가 유명한 분인가?? 암튼 그것보다 저 글이 왜 구걸이지? 난독증인가? 좋아요를 눌러달라거나 문상있으니 자기 채널에 들어오라는 말도 없구만',
 '진짜 게스트가 꼰채널 살리네ㅎ',
 '제 채널에 엄청난 보물이 있어요!!',
 

### 리스트 삭제
- 하나하나 삭제하면 인덱스를 찾기가 어렵다..  
- 하나 삭제하고 나면 인덱스가 바뀌니 인덱스와 출력을 같이 해주는 함수가 필요

In [22]:
def del_and_prt(li, idx):
    del li[idx]
    for idx, review in enumerate(li):
        print('인덱스: {}'.format(idx))
        print('\n')
        print(review)
        print('--------')
    return li

In [25]:
a = del_and_prt(temp_spam, 1)
a

인덱스: 0


학생,주부들 부업거리로👍👍👍
하루30분만 광고봐도 1만원
제 채널 영상확인하세요
--------
인덱스: 1


@하루1만원씩 챙겨가세요 채널클릭 관종2
--------
인덱스: 2


제 채널에 고추 길어지는법이 있습니다!~ 😘😘
--------
인덱스: 3


뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅
--------
인덱스: 4


제 채널에 오셔서 2만원 받아가세요~ 
1주일 남음 서두르세요!
(12월 31일까지)
--------
인덱스: 5


언니 유튜브 평생 해주세요 진짜 ... 레전드 채널 찾았움ㅠㅠ💖💖
--------
인덱스: 6


@네이버페이로5분만에 진짜 만원벌기 진용진 님 채널 다녀오셔요
--------
인덱스: 7


님 괜찮아여 힘들어도 힘내요~~~힘들면 쉬어도 되요 괜찮아요 저 이미 이 채널 있는 걸로 만족해요 힘들면 댓글로 말해주세여ㅎㅎ다음에 영상에서 봐여
--------
인덱스: 8


제가 잘 몰라서 그러는데 자기 채널에서 돈 받아가라는 사람들 채널 주인이 특정 텍스트 설정해서 그 텍스트는 댓글 입력 못하는거 없나요?
--------
인덱스: 9


근데 대단한건 런닝맨이 여기 채널도 그렇고 스브스 토렌트도 그렇고 런닝맨 관련 영상만 올리면 인기 급상승이 바로 돼냐 역시 대한민국을 대표하는 예능이다
--------
인덱스: 10


내 채널에 상해람보 있음 한번 봐보셈
--------
인덱스: 11


@이벤트알려주는채널 ㅈㄹ ㄴㄴ
--------
인덱스: 12


엇 저랑 아이디 똑같으심ㅋㅋㅋㅋㅋ너무 공감이요ㅜㅜ제 2의 개콘 채널이 되었으면..
--------
인덱스: 13


@이벤트알려주는채널 그만좀해라 넌
--------
인덱스: 14


지 채널 홍보를 왜 여기서해
--------
인덱스: 15


제 채널들어와봐용

공짜(돈/캐시/깊티)받아가세요
피해 ㄴㄴㄴ!
--------
인덱스: 16


@김수빈 글쓴이가 유명한 분인가?? 암튼 그것보다 저 글이 왜 구걸이지

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '@하루1만원씩 챙겨가세요 채널클릭 관종2',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '언니 유튜브 평생 해주세요 진짜 ... 레전드 채널 찾았움ㅠㅠ💖💖',
 '@네이버페이로5분만에 진짜 만원벌기 진용진 님 채널 다녀오셔요',
 '님 괜찮아여 힘들어도 힘내요~~~힘들면 쉬어도 되요 괜찮아요 저 이미 이 채널 있는 걸로 만족해요 힘들면 댓글로 말해주세여ㅎㅎ다음에 영상에서 봐여',
 '제가 잘 몰라서 그러는데 자기 채널에서 돈 받아가라는 사람들 채널 주인이 특정 텍스트 설정해서 그 텍스트는 댓글 입력 못하는거 없나요?',
 '근데 대단한건 런닝맨이 여기 채널도 그렇고 스브스 토렌트도 그렇고 런닝맨 관련 영상만 올리면 인기 급상승이 바로 돼냐 역시 대한민국을 대표하는 예능이다',
 '내 채널에 상해람보 있음 한번 봐보셈',
 '@이벤트알려주는채널 ㅈㄹ ㄴㄴ',
 '엇 저랑 아이디 똑같으심ㅋㅋㅋㅋㅋ너무 공감이요ㅜㅜ제 2의 개콘 채널이 되었으면..',
 '@이벤트알려주는채널 그만좀해라 넌',
 '지 채널 홍보를 왜 여기서해',
 '제 채널들어와봐용\n\n공짜(돈/캐시/깊티)받아가세요\n피해 ㄴㄴㄴ!',
 '@김수빈 글쓴이가 유명한 분인가?? 암튼 그것보다 저 글이 왜 구걸이지? 난독증인가? 좋아요를 눌러달라거나 문상있으니 자기 채널에 들어오라는 말도 없구만',
 '진짜 게스트가 꼰채널 살리네ㅎ',
 '제 채널에 엄청난 보물이 있어요!!',
 '@바두자 님이나 하고 이런 채널 와서 그런짓 하지 마세요',
 '와 탑현님 채널에도 댓썼는데...',
 '@하루 10,000원 받아가세요채널클릭 나 돈 많아!!!',
 '@mmm제채널 하루1만원 받아가세요mmm 조까',
 '역대급 bj 방송사

In [304]:
a = del_and_prt(a, 153)
a

인덱스: 0


학생,주부들 부업거리로👍👍👍
하루30분만 광고봐도 1만원
제 채널 영상확인하세요
--------
인덱스: 1


제 채널에 고추 길어지는법이 있습니다!~ 😘😘
--------
인덱스: 2


제 채널에 오셔서 2만원 받아가세요~ 
1주일 남음 서두르세요!
(12월 31일까지)
--------
인덱스: 3


제 채널들어와봐용

공짜(돈/캐시/깊티)받아가세요
피해 ㄴㄴㄴ!
--------
인덱스: 4


@하루 10,000원 받아가세요채널클릭 나 돈 많아!!!
--------
인덱스: 5


역대급 bj 방송사고 터진듯?
걸그룹 연습생 출신이라더라
제 채널 와서 보세요..
--------
인덱스: 6


제 채널 들어와봐용 

공짜(돈/캐시/깊티) 받아가세요 !
사기 ㄴㄴ
--------
인덱스: 7


제 채널에서 핵 코드 베포로 5분만에 때돈 버는 법 있어요. 볼거면 보세요.
--------
인덱스: 8


제 채널에 좋은거 있어요
--------
인덱스: 9


제 채널에 네.이버웹툰 쿠.키 200개 5분만에 무료로 받는법 올려뒀어요❤
--------
인덱스: 10


제 채널 들어와봐용 

공짜(캐시/돈/깊티) 받아가세용
피해 ㄴㄴ
--------
인덱스: 11


제 채널에 보면 큰일나는 영상있음
--------
인덱스: 12


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㄹㅁㄹㄹㅁㄹ응ㅇㅇ
--------
인덱스: 13


제 채널에서 꽁돈 받아가세요🥳🥳🥳🥳
--------
인덱스: 14


제 채널에 10분만 있으면 기프트카드도 계좌로도 돈벌수 있는 방법이 있습니다!
절 믿고 따라오실수 있겠습니까?
(친구가 많다면 돈을 더 벌수도 있답니다!)
--------
인덱스: 15


학생,주부들 용돈벌이로👍👍👍
하루 1만원씩 한달30공짜수준으로 가져가세요
제 채널에 영상 있어용♥


ㄹㅁㅁㄹㅁㄹㅁ
--------
인덱스: 16


제 채널에서 만원 받아가세
--------
인덱스:

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '제 채널들어와봐용\n\n공짜(돈/캐시/깊티)받아가세요\n피해 ㄴㄴㄴ!',
 '@하루 10,000원 받아가세요채널클릭 나 돈 많아!!!',
 '역대급 bj 방송사고 터진듯?\n걸그룹 연습생 출신이라더라\n제 채널 와서 보세요..',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊티) 받아가세요 !\n사기 ㄴㄴ',
 '제 채널에서 핵 코드 베포로 5분만에 때돈 버는 법 있어요. 볼거면 보세요.',
 '제 채널에 좋은거 있어요',
 '제 채널에 네.이버웹툰 쿠.키 200개 5분만에 무료로 받는법 올려뒀어요❤',
 '제 채널 들어와봐용 \n\n공짜(캐시/돈/깊티) 받아가세용\n피해 ㄴㄴ',
 '제 채널에 보면 큰일나는 영상있음',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㄹㅁㄹㄹㅁㄹ응ㅇㅇ',
 '제 채널에서 꽁돈 받아가세요\U0001f973\U0001f973\U0001f973\U0001f973',
 '제 채널에 10분만 있으면 기프트카드도 계좌로도 돈벌수 있는 방법이 있습니다!\n절 믿고 따라오실수 있겠습니까?\n(친구가 많다면 돈을 더 벌수도 있답니다!)',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㄹㅁ',
 '제 채널에서 만원 받아가세',
 '제 채널에 네이버.웹툰 쿠키 200개 5분만에 공짜로 받는법 올려뒀어요.❤',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㅁㄹㅁㅁ',
 '제 채널에 들어오셔서 문상 2마넌 가져가세엽!!\n거짓말 아니에여 ㅠㅠ 받아가시고 말하세요ㅠ',
 '

In [307]:
spam_list

['학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '제 채널에 고추 길어지는법이 있습니다!~ 😘😘',
 '제 채널에 오셔서 2만원 받아가세요~ \n1주일 남음 서두르세요!\n(12월 31일까지)',
 '제 채널들어와봐용\n\n공짜(돈/캐시/깊티)받아가세요\n피해 ㄴㄴㄴ!',
 '@하루 10,000원 받아가세요채널클릭 나 돈 많아!!!',
 '역대급 bj 방송사고 터진듯?\n걸그룹 연습생 출신이라더라\n제 채널 와서 보세요..',
 '제 채널 들어와봐용 \n\n공짜(돈/캐시/깊티) 받아가세요 !\n사기 ㄴㄴ',
 '제 채널에서 핵 코드 베포로 5분만에 때돈 버는 법 있어요. 볼거면 보세요.',
 '제 채널에 좋은거 있어요',
 '제 채널에 네.이버웹툰 쿠.키 200개 5분만에 무료로 받는법 올려뒀어요❤',
 '제 채널 들어와봐용 \n\n공짜(캐시/돈/깊티) 받아가세용\n피해 ㄴㄴ',
 '제 채널에 보면 큰일나는 영상있음',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㄹㅁㄹㄹㅁㄹ응ㅇㅇ',
 '제 채널에서 꽁돈 받아가세요\U0001f973\U0001f973\U0001f973\U0001f973',
 '제 채널에 10분만 있으면 기프트카드도 계좌로도 돈벌수 있는 방법이 있습니다!\n절 믿고 따라오실수 있겠습니까?\n(친구가 많다면 돈을 더 벌수도 있답니다!)',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㄹㅁ',
 '제 채널에서 만원 받아가세',
 '제 채널에 네이버.웹툰 쿠키 200개 5분만에 공짜로 받는법 올려뒀어요.❤',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㅁㄹㅁㅁ',
 '제 채널에 들어오셔서 문상 2마넌 가져가세엽!!\n거짓말 아니에여 ㅠㅠ 받아가시고 말하세요ㅠ',
 '

In [306]:
spam_list.extend(a)

In [308]:
spam_2 = get_KeywordDF('체널')
spam_2

['여기 답글 진짜 다 꼴보기싫네 제 체널와서 ~받아가세요~ ㅇㅈㄹ;;;;',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요\n\n😍😍😍😍😍😍😍😍',
 '제 체널에서 10000원 벌어가세요 ㅎㅎ',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등\n\n꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '@바두자왜 남에 체널에와서 홍보해요?     그리고 사기져?',
 '와..이거 군대휴가 나와서 체널돌리다 얼떨결에 본거같은데 그때는 와..신기하네~지금보니 아..그냥 눈물만ㅜㅜ그때나 지금이나 똑같은 영상인데 전해지는 느낌이 너무 다르다..',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊',
 '제 체널에 누구나 1분이면 총 5만원 얻는법 있어요! 진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '#이찬원  유튜브체널  민호님  영웅닝 희재님  #이찬원  닝축하해주시니 너무감사 해요.다들 좋은날들만 있으시길...항상 응원합니다.^^♡♡♡',
 '제 체널에 누구나 5.만.원 1분이면 얻는 방법 있습니다👍',
 '제 체널에 5.만.원 1분이면 얻는 방법 있습니다👍',
 '제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\nㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '코로나 이어려운시기  에유트부체널을통해  열심이 응원할께요     뿌잉 뿌잉    안 녕?사랑 해요~~~^^',
 '#이찬원   유튜브  체널축하합니다  회이팅 👍👍👍👍👍 트롯형님들찬원님사

In [325]:
b = del_and_prt(spam_2, 41)
b

인덱스: 0


제 체널에 간단히 꽁돈 버는법 3가지나 있어요!

진짜 있으니 해보고 말씀하세요

😍😍😍😍😍😍😍😍
--------
인덱스: 1


제 체널에서 10000원 벌어가세요 ㅎㅎ
--------
인덱스: 2


제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!

진짜 있으니 해보고 말씀하세요😄😄😶
--------
인덱스: 3


제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶
--------
인덱스: 4


제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등

꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 5


제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊
--------
인덱스: 6


제 체널에 누구나 1분이면 총 5만원 얻는법 있어요! 진짜 있으니 해보고 말씀하세요!!

기간 얼마 안남았대요ㅠㅠ

😍😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 7


제 체널에 누구나 5.만.원 1분이면 얻는 방법 있습니다👍
--------
인덱스: 8


제 체널에 5.만.원 1분이면 얻는 방법 있습니다👍
--------
인덱스: 9


제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!

진짜 있으니 해보고 말씀하세요!!

기간 얼마 안남았대요ㅠㅠ
ㅠ

😍😍😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 10


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!

😍😍😍😶😶😍😍😍😶😮😦😦😦😦
--------
인덱스: 11


제 체널에 누구나 30초면 2마넌, 웹툰 쿠키까지 받는법 올려놨어요...❤️
--------
인덱스: 12


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😶
--------
인덱스: 13


제 체널에 30초면 공짜로 3마넌 

['제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요\n\n😍😍😍😍😍😍😍😍',
 '제 체널에서 10000원 벌어가세요 ㅎㅎ',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등\n\n꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊',
 '제 체널에 누구나 1분이면 총 5만원 얻는법 있어요! 진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 5.만.원 1분이면 얻는 방법 있습니다👍',
 '제 체널에 5.만.원 1분이면 얻는 방법 있습니다👍',
 '제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\nㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n\n😍😍😍😶😶😍😍😍😶😮😦😦😦😦',
 '제 체널에 누구나 30초면 2마넌, 웹툰 쿠키까지 받는법 올려놨어요...❤️',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😶😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n진짜 있으니 해보고 말씀하세요\n😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😊😲',

In [326]:
spam_list.extend(b)

In [328]:
spam_3 = get_KeywordDF('챼널')
spam_3

['제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅌ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎ옹ㅎㅇ호오',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널오시면 아무나 5마넌 꽁짜로 얻는방법 올려놓았습니다!\n\n시간 얼마 남지 않았으니까 서둘러오세요\n\n깊카 문상 이딴거 아님 걱정 no no!\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음\n\n궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!\n\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!\n\n정말 진짜니까 안심하고 오세용😍😍',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이런거 절대 절대 아님\n\n😍★😍★😍★😍★

In [329]:
spam_list.extend(spam_3)

In [331]:
spam_4 = get_KeywordDF('30초')
spam_4

['제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음\n\n궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!\n\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!\n\n정말 진짜니까 안심하고 오세용😍😍',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등\n\n꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 챼널에 오셔서 30초만에 5마넌 가져가세요\n\n사기 진짜루 아니니까 믿고 오세요~~(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 믿고 오세요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '30초',
 '30초전 ㄷㄷ',
 '제 채널에 문화상품권 30초만에 받는 법 있음',
 '제 챼널오시면 30초만에 2마넌 얻는법 올려놓어요~~~~~ \n\n거짓말 진짜로 아니니까 놀러오세용!!!!!(12월31일까지 진행중)\n\n깊프트카드 문상 이딴거랑 차원이 다름.\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n\n😍😍😍😶😶😍😍😍😶😮😦😦😦😦',
 '제 챼널에 오셔서 30초만에 5마넌 획득하세요!\n\n사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '@김한나 영상이 30초겠조',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고

In [338]:
d = del_and_prt(spam_4, 42)
d

인덱스: 0


제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음

궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!

기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 1


제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!

정말 진짜니까 안심하고 오세용😍😍
--------
인덱스: 2


제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶
--------
인덱스: 3


제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!

진짜 있으니 해보고 말씀하세요😄😄😶
--------
인덱스: 4


제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등

꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 5


제 챼널에 오셔서 30초만에 5마넌 가져가세요

사기 진짜루 아니니까 믿고 오세요~~(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 믿고 오세요

😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★
--------
인덱스: 6


제 채널에 문화상품권 30초만에 받는 법 있음
--------
인덱스: 7


제 챼널오시면 30초만에 2마넌 얻는법 올려놓어요~~~~~ 

거짓말 진짜로 아니니까 놀러오세용!!!!!(12월31일까지 진행중)

깊프트카드 문상 이딴거랑 차원이 다름.

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥

😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥
--------
인덱스: 8


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!

😍😍😍😶😶😍😍😍😶😮😦😦😦😦
--------
인덱스: 9


제 챼널에 오셔서 30초만에 5마넌 획득하세요!

사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) 

깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!

😍♥😍♥😍♥

['제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음\n\n궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!\n\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널 오시면 아무나 30초만에 5마넌 공짜로 얻을수있어용!!!\n\n정말 진짜니까 안심하고 오세용😍😍',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 30초면 공짜로 3마넌 얻는법, 네이버페이 2마넌 받는법 등등\n\n꿀정보 여러가지 있어요!! 진짜있으니 해보고 말씀하세요!!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 챼널에 오셔서 30초만에 5마넌 가져가세요\n\n사기 진짜루 아니니까 믿고 오세요~~(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 믿고 오세요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 채널에 문화상품권 30초만에 받는 법 있음',
 '제 챼널오시면 30초만에 2마넌 얻는법 올려놓어요~~~~~ \n\n거짓말 진짜로 아니니까 놀러오세용!!!!!(12월31일까지 진행중)\n\n깊프트카드 문상 이딴거랑 차원이 다름.\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n\n😍😍😍😶😶😍😍😍😶😮😦😦😦😦',
 '제 챼널에 오셔서 30초만에 5마넌 획득하세요!\n\n사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '@김한나 영상이 30초겠조',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😶

In [339]:
spam_list.extend(d)

In [340]:
spam_5 = get_KeywordDF('하루')
spam_5
### 이걸로는 무리다

['@하루1만원씩 챙겨가세요 채널클릭 빅현배 저 방송후에 팬들이 제발 먹지말라해도\n강퇴시키고 기어코 파파존스 피자에 스파게티, 콜라 먹음 ㅋㅋㅋ\n유튜브엔 안올리고 아프리카에서만 먹는다고ㅋㅋ 나참',
 '학생,주부들 부업거리로👍👍👍\n하루30분만 광고봐도 1만원\n제 채널 영상확인하세요',
 '@하루10,000원 받아가세요 채널클릭 한번에 1억 줄 수 없다면 꺼지세여',
 '@하루10,000원 받아가세요 채널클릭 ㄲㅈ 제발',
 '@하루 10,000원 받아가세요채널클릭ㅇ ㅕ8ㅗ',
 'ㅋㅋㅋ 재밋게 잘만들었네요~ㅋ 개인적인 생각은 이 어린친구에게 역사관이나 정치적인 질문은 안했으면 하네요 :) 어리기도 하고 일본에 살고있는 한국인이라.. 어떤대답을 해도 욕먹을수있는 포지션이기에ㅠ ㅠ 물론 한국인이니 한국에 대한 올바른 역사관을 갖는건 중요하지만 지금은 지금을 즐기는게 더 나아보일 나이잖아요ㅎ  고딩브이로그에서 대딩 직딩 엄마 할머니 브이로그까지 이어지길 기대할께요\nPS 영상은 하루에 한번씩 부탁드립니다ㅋㅋ 있는건 다 쓰고 나중에 본업과 유투버 사이를 고민하실때 영상업로드 주기를 결정합시다ㅋㅋ',
 '오늘도 내일도 행복한 하루 되십쇼',
 '맘도착하고이쁜쯔양~\n상처받지말고일하시길.\n전진짜먹방보면서하루스트레스날립니다 \n고마워요~♡',
 '지금 보러 와야 진정한 깡팸입니다\n어떻게 살면서 한 입으로 두 말하겠습니까\n당신은 지인들에게 난 하루에\nN깡 한다며 자랑하거나\n깡팸이라고 말하고 다녔을 것입니다\n하지만 유행이 지났다고 이렇게 버리다면\n당신은 진정한 깡팸이 아닙니다\n정말 비를 좋아하거나 깡을 좋아한다면\n꼭 1일 1깡 이상 하셔야합니다\n지인들에게 깡을 다시 전파하시고\n제 2의 깡 신드롬을 일으킵시다!!!',
 '민정님 오늘 영상 빨리빨리 진행돼서 너무 좋습니다..❤️ 몰랐던 사실 큐엔에이 영상으로 잘알고갈게요 하루하루 민정님 영상 목빠지게 기다리고 있어요...\U0001f97a\U0001f97a 오늘도 너무 이쁘시고 남은 2020년 잘보내시

In [341]:
spam_6 = get_KeywordDF('누구나')
spam_6

['제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅌ',
 '옛날에 김완선이 아주 관능 적으로 율동적으로 춤추면서 노래를  매역적으 로 불럿다 지금은 누구나 너무도 춤을잘 추는갓다👏👏👏👏👏👏👏',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎ옹ㅎㅇ호오',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '@haenseom kim 니네 가족들한테 동남아 사람 같다거나 흑인 혼혈이라고 하면 감사합니다~라고 하세요 ㅎ \n요즘 애완 동물을 반려동물이라고 칭하며 정말 가족과 같이 키우시는 분들도 많은데\n강아지를 올바르게 키우기 위해 교육시키고 환경을 조성하는 노력도 마다하지 않는 주인이란걸 조금만\n봐도 알 수 있는 상황에 진도개 혈통이란걸 누구나 아는 상황인데 잡종이라고 한다? ㅋㅋ 가정교육 제대로 받은 사람이 맞니? \n피부 조금 까맣다고 동남아 사람같다느니 짓걸여도 된다는 말이지? ㅋㅋㅋ\n내가 볼 땐 니도 정신 나간새낀거 같은데',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 프로필에 누구나 5.만.원 1분이면 버는 방법 있어요👍👍👍진짜 

In [354]:
e = del_and_prt(spam_6, 58)
e

인덱스: 0


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅌ
--------
인덱스: 1


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ
--------
인덱스: 2


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎ옹ㅎㅇ호오
--------
인덱스: 3


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ
--------
인덱스: 4


제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!

진짜 있으니 해보고 말씀하세요😄😄😶
--------
인덱스: 5


제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶
--------
인덱스: 6


제 프로필에 누구나 5.만.원 1분이면 버는 방법 있어요👍👍👍진짜 있으니 해보고 말씀하세요😍😍
😍
--------
인덱스: 7


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅇㅈ
--------
인덱스: 8


제 채널에 네이버웹툰 쿠키 200개 누구나 5분안에 공짜로 받는 법 올려뒀어요❤
--------
인덱스: 9


제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎ

['제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅌ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅎㅎ',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎ옹ㅎㅇ호오',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅎㅇㅋㅋㄹㅇ',
 '제 체널에 누구나 쉽게 30초면 2마넌, 웹툰 쿠키 200개 얻는법 있어요!!\n\n진짜 있으니 해보고 말씀하세요😄😄😶',
 '제 체널에 누구나 쉽게 30초면 3마넌 버는법 있어요!!!진짜 있으니 해보고 후기남겨주세요😊😶😶',
 '제 프로필에 누구나 5.만.원 1분이면 버는 방법 있어요👍👍👍진짜 있으니 해보고 말씀하세요😍😍\n😍',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㅇㅈ',
 '제 채널에 네이버웹툰 쿠키 200개 누구나 5분안에 공짜로 받는 법 올려뒀어요❤',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!❤😃❤😃❤😍❤😍❤ 진짜 거짓말 아니에요!! 해보고 말해요 우리ㅎㅎ!!!!!12월31일에 이벤트 끝나니까 서두르자!!!❤❤❤ㄹㅇ',
 '제 체널에 누구나 쉽게 꽁돈 얻는법 3가지나 있어요!!!진짜 있으니 해보고 후기남겨주세요😊',
 '제 챼널에 진짜 정말 누구나 쉽게 2마넌 받는법 있습니다!!!!

In [355]:
spam_list.extend(e)

In [356]:
len(spam_list)

358

In [357]:
len(list(set(spam_list)))

255

In [358]:
spam_6 = get_KeywordDF('용돈')
spam_6

['이게 진짜 총몇명이 잼민이들한테 폭발적인가봐 용돈줬더니 총몇명 책사왔어ㅋㅋㅋㅋ',
 '@이민정 너네엄마 용돈 ㄱ',
 '제 채널 들어와서 쉽게 용돈버세요! 이런거 관심 없으신 분들에겐 죄송합니다',
 '٩(*•̀ᴗ•́*)و♥ ㅊH널클릭 ♥٩(*•̀ᴗ•́*)و\n♥하루30분 광 고 보 고♥\n♥매일 용돈 1만씩 벌어가세요♥\n.\n오늘도 즐거운하루되세요!! ♥(۶•౪•)۶ \n\nㄹㅁㄹㅁㄹㅁㄹㅁㄹㅁㄹㅁ\nㄹㅁㅁㄹㅁ',
 '🌟 다들 여기야 여기 🌟\n\n\n1. 이름 0:33\n\n2. 생년월일 0:35\n\n3. 나이 0:38\n\n4. 키/몸무게 0:42\n\n5. mbti 0:49\n\n6. 국적 0:54\n\n7. 부모님 0:56\n\n8. 형제 1:02\n\n9. 인스타 아이디 1:09\n\n10. 남자친구 + 설레는 썰 1:23\n\n11. 짝사랑 1:43\n\n12. 졸업후의 계획 1:59\n\n13. 유튜브를 시작한 계기 2:06\n\n14. 메이크업 2:19\n\n15. 이상형 2:30\n\n16. 머리 고데기 2:52\n\n17. 좋아하는 배우 아이돌 2:59\n\n18. 학교 성적 3:03\n\n19. 한국 재귀국 3:06\n\n20. 좋아하는 일본 음식 3:16\n\n21. 한국가면 하고 싶은거 3:19\n\n22. 한국이 그리울때 3:29\n\n23. 한국음식 그리워? 3:34\n\n24. 옷 구매처 3:43\n\n25. 한국에 거주할때 어디 살았는지 3:47\n\n26.꿈 생각은 어느나라 말? 3:50\n\n27. 일본에서 쓰는 이름 4:05\n\n28. 편한 언어 4:17\n\n29. 핸드폰 언어 4:26\n\n30. eju? JLPT? 4:31\n\n31. 친구들이 주로 쓰는 sns 4:41\n\n32. 지금 친한 친구랑 친해진 계기 4:52\n\n33. 일본에서 처음사귄 친구 4:58\n\n34. 혼혈친구 많은지 5:02\n\n35. 앞머리 내린 애들이 많은 이유 5:07\n\n36. 친구들 다 쌍커풀 있는 이유 5:16\n\n3

In [361]:
df = pd.DataFrame(spam_list, columns=['spams'])

In [362]:
import pickle

In [363]:
df.to_pickle('스팸리스트_1.pkl')

In [24]:
spam_7 = get_KeywordDF('꽁돈')
spam_7

['제 채널에 오셔서 꽁돈15000원받아가세요ㅎㅎ\n1분안에15000원입니다!!  😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😀😀😀😀😀👈👈👈👈👈👈👈👈👈',
 '@딸써니Dalsunny 박보람이라고 딴 댓에서도 계속 꽁돈 받으러 오세요~~ 하는 사람있어서 저렇게 말했는데 댓삭 하고 튄듯요 ㅋㅋㅋㅋ',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요\n\n😍😍😍😍😍😍😍😍',
 '제 채널에서 꽁돈 15000원받는법있어요!',
 '@COVID-19현황 알리미 그래도 5000원 꽁돈 ㅇㅈㄹ 하는 애들보단 낫다',
 '제 체널에 간단히 30초면 3마넌 얻는법 등\n\n꽁돈 얻는법이 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '1분만에 꽁돈 7천원 버는법 재 채널 오시면 있음!',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n진짜 있으니 해보고 말씀하세요\n😍😍😍😍😍😍😍😍',
 '제 채널에서 꽁돈 받아가세요\U0001f929\U0001f929\U0001f929\U0001f929']

In [25]:
spam_8 = get_KeywordDF('공짜')
spam_8

['@꼰대희 \n학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥',
 '🤗제 영상에 문상 5000원올려놨어여!🤗\n\n\n필요하신분 있으면 공짜로 받아가셔용^0',
 '🤗제 영상에 문상 5000원 올려놨어여!🤗\n\n\n필요하신분 있으면 공짜로 받아가셔용^0',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㄹㅁㄹㅁㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㄹㄹㅁㅁㄹㅁㅁㄹ',
 '제 챼널 오시면 누구나 30초만에 5마넌 공짜로  가져갈수있어요~\n\n정말 진짜니까 안심하고 오세용😍😍',
 '제 채널들어와봐용\n\n공짜(돈/캐시/깊티)받아가세요\n피해 ㄴㄴㄴ!',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n\nㄹㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㅁㅁㅁㄹㅁㄹㄹㅁㄹㅁㄹㅁㄹㅁㅁ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\n로롬ㄹㅁㄹㅁㄹㅁㄹㅁ',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 챼널 오시면 누구나 30초만에 5마넌 공짜로  가져갈수

In [26]:
spam_9 = get_KeywordDF('문상')
spam_9

['그래도 문상년놈들보단 낫지',
 '@박진우 차라리 저런게 낫지 ㅋㅋ\n??:제 채널에오셔서 공짜 문상받아가세요!\n이거보단 좋은의미잖어 ㅋㅋ',
 '수입 들어오면 깊카나 문상으로 돈뿌립니다 구독 부탁드려요',
 '🤗제 영상에 문상 5000원 올려놨어여!🤗\n\n\n필요하신분 있으면 공짜로 받아가셔용0',
 '🤗제 영상에 문상 5000원올려놨어여!🤗\n\n\n필요하신분 있으면 공짜로 받아가셔용^0',
 '🤗제 영상에 문상 5000원 올려놨어여!🤗\n\n\n필요하신분 있으면 공짜로 받아가셔용^0',
 '제 채널에 문상 꽁으로 받는법 있어요! 받아가세요!',
 'https://youtu.be/ElMIg941zVs\n↑댓글만 달면 문상 2만원이 무료?!',
 '제 챼널오셔서 30초만에 5마넌 가져가세요!\n\n이런걸로 거짓말 안합니다! 언능 오세요(12월31일까지 진행중)\n\n깊프트카드 문상 이딴거랑 차원이 다름!!!!!!!!!!!!!\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 채널에 문상 5천원 꽁으로 받는법 있어요!!',
 '제 챼널오셔서 30초만에 5마넌 가져가세요!\n\n이런걸로 거짓말 안합니다! 언능 오세요(12월31일까지 진행중)\n\n깊프트카드 문상 이딴거랑 차원이 다름!!!!!!!!!!!!!\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '제 챼널오셔서 30초만에 5마넌 가져가세요!\n\n이런걸로 거짓말 안합니다! 언능 오세요(12월31일까지 진행중)\n\n깊프트카드 문상 이딴거랑 차원이 다름!!!!!!!!!!!!!\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '이야 유튜브 어쩌다 이렇게됬냐 한 뎃글에 낚시빌런, 광고빌런, 문상빌런, 대깨문 다있냐 그것도 몇명씩이나']

In [27]:
spam_10 = get_KeywordDF('꽁으로')
spam_10

['제 채널에 문상 꽁으로 받는법 있어요! 받아가세요!', '제 채널에 문상 5천원 꽁으로 받는법 있어요!!']

In [29]:
spam_11 = get_KeywordDF('만원')
spam_11

['@네이버페이로5분만에 진짜 만원벌기 밑에 3만원보다 딸리는데 노력하십시오',
 '@10분만에 만원을 버는 간단한 방법 넌 또뭐야 걍 꺼져',
 '@꼰대희 \n학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😶😮😦',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😊😲',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㄹㅁㄹㅁㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ',
 '@네이버페이로5분만에 진짜 만원벌기 잼민인데요',
 '양팡 복귀함 ㄹㅇ 구라면 100만원 지른 롤계정 삭제https://youtu.be/4lyLu91rjx4 .',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㅁㄹㄹㄹㅁㅁㄹㅁㅁㄹ',
 '@김한나 이 분한테 5만원 사기 당함',
 '@mmm제채널 하루1만원 받아가세요mmm 이딴거 달리는건 나 베댓급으로 쳐주는거지않냐ㅋㅋㅋㅋ 영광이구만',
 '@10분만에 만원을 버는 간단한 방법 얘기 꺼내자마자 귀신 같이 나타나네 ㅋㅋㅋㅋㅋㅋㅋ',
 '제 채널에 오셔서 2만원 받아가세요~\n2일 남았어요!! 서두르세요 ㄹㅇ줍니다\n(12월 31일까지)',
 '만원이 사라지는 마술 너무 재밌게 잘봤어요 ㅋㅋㅋㅋㅋ 그리고 진짜 신기했던건...대희형님 주량이........저는 최소 5병은 드시는줄알았는데.....놀랐습니다.....잘보고가요~~',
 '천만원 빌리기에서 냉큼 빌려준다는 거 보고 권재관 감동이네 싶었음...김대희보다 힘들 수도 있을 것 같은데ㅠㅠ',
 '출연료 만원 싸네요 ㅋㅋㅋㅋ',
 '@이벤트저장소 5만원획득=이상한 듣보 사이트 5만포인트를 칭함.\n결국 추천인 누르면 지한테만 유리한 수법임.\n괜히 헛수고

In [30]:
spam_12 = get_KeywordDF('마넌')
spam_12

['제 챼널오면 누구나 5마넌 꽁짜로 얻는방법 있어요~\n\n선착순이니까 서두르세용~(12월31일까지 진행중) \n\n기프트캬드, 문화샹품권 이런거 아닙니다. 믿고오세용\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널 오시면 누구나 30초만에 5마넌 공짜로  가져갈수있어요~\n\n정말 진짜니까 안심하고 오세용😍😍',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 챼널에 아무나 가입 5마넌 얻는법 올려놨음\n진짜 거짓말 절대로 아니니니까 믿고오세요\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널 오시면 누구나 30초만에 5마넌 공짜로  가져갈수있어요~\n\n거짓말 아니니까 믿고 오세요😍😍',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음\n\n궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!\n\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널에 오셔서 30초만에 5마넌

In [31]:
spam_13 = get_KeywordDF('거짓말')
spam_13

['거짓말 탐지기는 왜갔다놧어 ㅋㅋㅋㅋ',
 '제 챼널에 아무나 가입 5마넌 얻는법 올려놨음\n진짜 거짓말 절대로 아니니니까 믿고오세요\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널 오시면 누구나 30초만에 5마넌 공짜로  가져갈수있어요~\n\n거짓말 아니니까 믿고 오세요😍😍',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널에 오셔서 30초만에 5마넌 획득하세요!\n\n사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '@기니적도 문재인 대통령 사랑합니다 \n🤚 반박시 토착왜구 일베새끼 박제 🤚\n👊검찰개혁👊조국수호👊언론개혁👊\n\n\n\n\n\n\n\n\n\n\n\n여러분 이거 다 거짓말인거 아시죠?',
 '제 챼널오셔서 30초만에 5마넌 가져가세요!\n\n이런걸로 거짓말 안합니다! 언능 오세요(12월31일까지 진행중)\n\n깊프트카드 문상 이딴거랑 차원이 다름!!!!!!!!!!!!!\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널오셔서 30초만에 5마넌 가져가세요!\n\n이런걸로 거짓말 안합니다! 언능 오세요(12월31일까지 진행중)\n\n깊프트카드 문상 이딴거랑 차원이 다름!!!!!!!!!!!!!\n\n😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥😍♥',
 '제 챼널에 오셔서 30초만에 5마넌 획득하세요!\n\n사기 아

In [32]:
spam_14 = get_KeywordDF('프로필')
spam_14

['제 프로필에 누구나 5.만.원 1분이면 버는 방법 있어요👍👍👍진짜 있으니 해보고 말씀하세요😍😍\n😍',
 '\U0001f973제 프로필 오셔서 ㅁㅅ.ㄱㅋ 5만원권 바로 받아가세요╥﹏╥。◕‿◕ 。(❛◡❛)✿ᕙ(•̀‸•́‶)ᕗ',
 '#이찬원 늘 응원하고 사랑합니다^^ 대문 예쁘게 꾸며주신 분께 감사드립니다. 배경은 좋은데 동그라미 안에는 찬원님 실물사진 넣어주세요~^^  너무 아기같은 느낌이라서 프로필 실물사진으로 바꿔주세요. 부탁드려요~~',
 '유트브 프로필이랑 이분이랑 똑같은...분?',
 '찬원오빠 프로필 사진 바꾸셨네용~ 넘넘 귀여워요 ㅎㅎ 글구 영상속에서 이찬원TV개국했다고 하니까 "와아아아~"라고 말하는 희재?오빠도 귀여워 ㅋㅋ',
 '찬또배기 유튜브 프로필 넘 귀여워졌네요ㅋㅋㅋ 그 공약들 꼭 지키시길...!😁😁',
 '#이찬원유튜브 프로필사진을 찬원님 실물사진으로 바꿔주세요~ 솟대배경은 좋은데 사진이 너무 유아틱하고 만화적입니다. 동그라미 안에만 이찬원 실물사진을 넣어주시면 좋겠어요. 대문 꾸며주신분께 감사드립니다^^',
 '제 프로필에 누구나 5.만.원 1분이면 버는 방법 있습니다👍',
 '너무 좋으시겠어요 저두 정동원군 너무좋아하는데 딸이 올해생일선물로 동원이 앨범하고 사진을선물로해줘서받고 폰 프로필사진도 동원이사진으로해놨답니다',
 '#이찬원_유튜브_프로필사진은 이찬원 사진으로 올려주면 더 좋을것같아요 대문사진도 프로필 사진도 귀엽긴한데 너무 유아용같은 느낌이 살짝 드네요',
 '#이찬원 님 영상 꼬마웡,,\n형님들 응원감사해요.\n배너랑 프로필도 너무 귀여워용',
 '유튜브프로필사진넘 귀여워요\n30만가자~고고고',
 '\U0001f973제 프로필 오셔서 ㅁㅅ,ㄱㅋ 5만원권 바로 받아가세요。◕‿◕ 。╥﹏╥。◕‿◕ 。╥﹏']

In [33]:
spam_15 = get_KeywordDF('문화')
spam_15

['사회 생활 하다보면 나이가 어려보이거나 체구가 왜소해 보이면  반말을 은연중에 하는 사람들이 있지!    꼰대문화 권위적인 문화는 청산해야 한다..  남자꼰대만 있는게 아니라 여자꼰대도 있다...  찐 ㄹㅇ ㅠㅠ',
 '제 챼널오면 누구나 5마넌 꽁짜로 얻는방법 있어요~\n\n선착순이니까 서두르세용~(12월31일까지 진행중) \n\n기프트캬드, 문화샹품권 이런거 아닙니다. 믿고오세용\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 채널에 문화상품권 30초만에 받는 법 있음',
 '제 챼널에 아무나 가입 5마넌 얻는법 올려놨음\n진짜 거짓말 절대로 아니니니까 믿고오세요\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널에 오면 30초만에 5마넌 얻는법 올려놨음\n\n궁금하면 일단 믿고 오셔요~ 사 기 절 대 아 님!\n\n기프트캬드, 문화샹품권 이런거 아니니까 믿고 오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '꼰대문화는 개뿔... 나이처먹고 중고딩때 하던 양아치짓 하는거지',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '제 챼널에 오셔서 30초만에 5마넌 획득하세요!\n\n사기 아니고 거짓말 아니니까 믿구오세요(12월31일까지 진행중) \n\n깊카 문화샹품권 이딴거랑 차원이 다릅니다. 얼른 오세요!\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '꼰대님 님이 욕파하면 후배는  문화가 됩니다.//!

In [36]:
spam_16 = get_KeywordDF('3마')
spam_16

['제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 체널에 간단히 30초면 3마넌 얻는법 등\n\n꽁돈 얻는법이 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍']

In [37]:
spam_17 = get_KeywordDF('3만')
spam_17

['@네이버페이로5분만에 진짜 만원벌기 밑에 3만원보다 딸리는데 노력하십시오',
 '이걸 아직도 몰라?\n\n"인싸잡"\n나이/지역/장소 X\n 합법적인 간단알바\n 하루 3만원\n카카오톡 상단 돋보기 클릭!!\n""인싸잡""검색',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😶😶😶',
 '뭐야 조회수가 133회인줄알았는데 133만회네 성공했네 채널 ㅋ 앞으로도 화이팅',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😶😮😦',
 '재미있다~\n\n"인싸잡"\n나이/지역/장소 X\n 합법적인 간단알바\n 하루 3만원\n카카오톡 상단 돋보기 클릭!!\n""인싸잡""검색',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😊😲',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😊',
 '@박보람  3만 원밖에 안 줘요?']

In [38]:
spam_18 = get_KeywordDF('영상')
spam_18

['@꼰대희 \n학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥',
 '앞으로 좋은 영상 부탁드리겠습니다',
 '🤗제 영상에 문상 5000원 올려놨어여!🤗\n\n\n필요하신분 있으면 공짜로 받아가셔용^0',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㄹㅁㄹㅁㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ',
 '매일보고 있어요 너무재미있어요💕앞으로 더 재미있는 영상올려 주세요 ㅎㅎ',
 '0:16 웃음참는게 보임 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ \n영상 다보고나니까 복선이었어 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '영상 잘보고갑니다.\n항상 응원합니다.',
 '@밀크초코 제발 여기는 서준이모음 영상이에요 ㅠㅠ 제가 수호모음에서 그러면 좋아요? 아까 댓글에서도 그러더니ㅠㅠ (수호파가 여기 오지 말라는 게 아님 오해 ㄴㄴ) 서준파는 서준이 분량 존나 적어서 짜증나는데 이런 모음집이 얼마나 귀한건지 ㅠㅠ',
 '속이 다시원하네   \n어줍짰는 동네 꼰대들 하는짓은 전국팔도 조선인 종특인가 보구려 \n영상처럼 두어번 해본적 있는데 티격태격 하고도 다음날이면 멀쩡하게 돌아댕기는거 보면 열불이 납디다 \n암튼 재미난 영상이고 코로나이후엔 손님도 에티켓을  가진 세상이 되길 바래봅니다',
 '인기 급상승 동영상 1위 축하드려요🎉🎉',
 '우리 동원이 사랑해주시고 응원해주셔서 눈물나게 감사합니다\n우리 동원이 보러 동원네 카페 가고싶으신데 참으신다는 영상 예전에 봤어요\n동원이를 위하는 진심이 느껴졌어요\n그리고 우리 동원이의 노래와 연주로 할머니들이 위로 받으며 즐거워하는 모습 같이삽시다에서 보고싶어요 그날이 올때까지 응원하며 기다릴게요\n앞으로도 우리 동원이 응원 많이 부탁드려요',
 '넘 쟈밌어ㅓ맨날 영상 끝까지 보는데 이대로 가다간 족보 꼬일거같엉ㅋㅋㅋ',
 '인기동영상 1위 축하행',
 "'긴대희'로 무편집 영상 올려주세요!!!",
 '핰ㅋㅋㅋㅋㅋㅋㅋㅋ 이게 급상승 동영상 2위얔ㅋㅋ

In [52]:
spam_19 = get_KeywordDF('이벤트')
spam_19

['@이벤트저장소 여기서는 너가 짱이다 ㅋㅋ',
 '@이벤트저장소 그만좀 하세요 화나게 하지 말고요 그딴거에 관심 없어요',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '@이벤트저장소 싫어요!!!!!!!😍☆😍☆',
 '@이벤트저장소 5만원획득=이상한 듣보 사이트 5만포인트를 칭함.\n결국 추천인 누르면 지한테만 유리한 수법임.\n괜히 헛수고하는 호구 업기를\n+이런 ㅈ같은 광고하는 사기꾼들 댓글 옆에 동그라미3개 클릭후 스팸 및 상업적 광고 신고 ㄱㄱㄱㄱ',
 '@이벤트저장소 님이나 많이 가지셈',
 '혹시 이벤트로 먹방같은거 찍어주실 생각없나요? ;;;',
 '제 챼널오면 5마넌 얻는법 올려놓음. \n\n거짓말 진짜아님. 이벤트 종료 되기전에 와서 직접 해보고 받아가세용\n\n기프트캬드 문화샹품권 이딴거랑 차원이 다르니까 믿고오셔요\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★\n\n😍★😍★😍★😍★😍★😍★😍★😍★😍★😍★',
 '@이벤트저장소 개소리임',
 '@이벤트저장소 진짜 민폐..',
 '제 채널에 합 4만원받을수 있는 이벤트 올려놨어요~      1']

# 스팸 추출

In [281]:
spam_2 = b

In [269]:
b = del_and_prt(b, 13)
b

인덱스: 0


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😶😮😦
--------
인덱스: 1


제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!
😍😍😍😶😶😍😍😍😊😲
--------
인덱스: 2


제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 3


제 체널에 간단히 꽁돈 버는법 3가지나 있어요!
진짜 있으니 해보고 말씀하세요
😍😍😍😍😍😍😍😍
--------
인덱스: 4


제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 5


제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!
진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 6


제 체널에 누구나 1분이면 총 5만원 얻는법 있어요! 진짜 있으니 해보고 말씀하세요!!

기간 얼마 안남았대요ㅠㅠ

😍😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 7


제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!

진짜 있으니 해보고 말씀하세요!!

기간 얼마 안남았대요ㅠㅠ
ㅠ

😍😍😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 8


제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!

진짜 있으니 해보고 말씀하세요!!

기간 얼마 안남았대요ㅠㅠ

😍😍😍😍😍😍😍😍😍😍😍😍
--------
인덱스: 9


제 체널에 무료로 5000원 받을수있는 방법있어요


꼭 참여하셔서 받아가세요~~~😋
--------
인덱스: 10


제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!

진짜있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍😍
--------
인덱스: 11


제 체널에 간단히 30초면 3마넌 얻는법 등

꽁돈 얻는법이 3가지나 있어요!

진짜 있으니 해보고 말씀하세요!!

😍😍😍😍😍😍😍😍

['제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😶😮😦',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😊😲',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n진짜 있으니 해보고 말씀하세요\n😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 1분이면 총 5만원 얻는법 있어요! 진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\nㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 간단히 누구나 1분이면 총 5만원 얻는법 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n기간 얼마 안남았대요ㅠㅠ\n\n😍😍😍😍😍😍😍😍😍😍😍😍',
 '제 체널에 무료로 5000원 받을수있는 방법있어요\n\n\n꼭 참여하셔서 받아가세요~~~😋',
 '제 체널에 30초면 공짜로 3마넌 얻는법 있어요!!\n\n진짜있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 체널에 간단히 30초면 3마넌 얻는법 등\n\n꽁돈 얻는법이 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요!!\n\n😍😍😍😍😍😍😍😍😍',
 '제 체널에 누구나 3만원 30초면 얻는법 있어요!!!진짜 있으니 해보고 말씀하세요!!!\n😍😍😍😶😶😍😍😍😶😶😶',
 '제 체널에 간단히 꽁돈 버는법 3가지나 있어요!\n\n진짜 있으니 해보고 말씀하세요

In [270]:
spam_list.extend(b)

In [280]:
spam_3 = c

In [273]:
spam_list.extend(c)

In [276]:
# 이걸로 걸러내기는 어렵다.
spam_4

['@하루 10,000원 받아가세요채널클릭 \n꺼져줄래?',
 '찬원님#  왕자님 팬앤스타 1위 축하드림니다 찬워님 저희에게 와주셔서 감사하고 행복합니다 하루를 찬원님 과 하루를 시작합니다 "내게 애인이 생겼어요 아주그냥 찬원님 왕자님께 푹우욱 빠져버렸습니다 " 찬원님 화이팅 🙌 찬원님 응원합니다  21년도에도 찬원님 항상 반짝반짝 빛나는 한해되시고 축복 많이 받고 세계로 가즈아 🤴🏻💛🌈💝👍💐💞👏',
 '지금 보러 와야 진정한 깡팸입니다\n어떻게 살면서 한 입으로 두 말하겠습니까\n당신은 지인들에게 난 하루에\nN깡 한다며 자랑하거나\n깡팸이라고 말하고 다녔을 것입니다\n하지만 유행이 지났다고 이렇게 버리다면\n당신은 진정한 깡팸이 아닙니다\n정말 비를 좋아하거나 깡을 좋아한다면\n꼭 1일 1깡 이상 하셔야합니다\n지인들에게 깡을 다시 전파하시고\n제 2의 깡 신드롬을 일으킵시다!!!',
 '오늘도 내일도 행복한 하루 되십쇼',
 '@꼰대희 \n학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥',
 '저요..하루종일 황인엽 영상만 봄..저진짜 미친것 같아요 ㅋㅋ',
 '하루에 영상 2-3개씩 올라와도 괜찮으니깐 마구마구 와구와구 올려주세요!!!!',
 '#이찬원 님 만나고 제생활이 완전 바켜버렸어요  하루왠종일 손에서 폰이떠나질 않네요 요즘 집콕생활인데 찬원님과 함께라 견딜수 있는것 같아요 영원히 스릉합니다',
 '하루만에 조회수 140만 ㄷ',
 '학생,주부들 용돈벌이로👍👍👍\n하루 1만원씩 한달30공짜수준으로 가져가세요\n제 채널에 영상 있어용♥\n\n\nㄹㅁㄹㅁㄹㅁㅁㄹㅁㄹㅁㄹㅁㅁㄹㅁ',
 '보는 저 까지도 다 기분이 좋아질 수 있게 행복 공유해주셔서 감사합니다. 말씀하시는 한 마디 한 마디 전부 다 너무 너무 공감이 가네요. 사랑할 수 밖에 없는 우리 예쁜 동원이♡♡♡ 나중에 꼭 박원숙님과 동원이가 함께 만나 방송하실 수 있는 날이 하루 빨리 왔으면 좋겠네요. 두 분 함께 있는 모습 꼭 보고 싶어요~ 날

In [ ]:
c = del_and_prt()

# 이모티콘
- 포함한 경우와 포함하지 않은 경우
- 딥러닝 vs 머신러닝 (표본을 많이 찾을수가 없다)